In [113]:
import json
import pdb

from brick_parser import tagsetList as tagset_list, \
                         tagList as tag_list,\
                         pointTagsetList as point_tagset_list
#from imp import reload
#reload(building_tokenizer)
from building_tokenizer import get_unit_dict, get_bacnettype_dict


In [114]:
unit_dict = get_unit_dict(building_name)
bacnettype_dict = get_bacnettype_dict(building_name)

In [72]:
def leave_one_word(s, w):
    if w in s:
        s = s.replace(w, '')
        s = w + '-' + s
    return s
def _bilou_phraser(sentence, token_labels):                                     
    phrase_labels = list()                                                  
    curr_phrase = ''                                                        
    for i, (c, label) in enumerate(zip(sentence, token_labels)):
        if 'right_identifier' in label:
            if 'right_identifier' not in curr_phrase and curr_phrase!='':
                phrase_labels.append(curr_phrase)                           
                curr_phrase = ''
            curr_phrase += 'right_identifier'+c
            continue
        elif 'left_identifier' in label:
            #pdb.set_trace()
            if 'left_identifier' not in curr_phrase and curr_phrase!='':
                phrase_labels.append(curr_phrase)
                curr_phrase = ''
            curr_phrase += 'left_identifier'+c
            continue
        tag = label[0]                                                      
        if tag=='B':                                                        
            if curr_phrase:                                                 
            # Below is redundant if the other tags handles correctly.       
                phrase_labels.append(curr_phrase)                           
            curr_phrase = label[2:]                                         
        elif tag=='I':                                                      
            if curr_phrase != label[2:]:                                    
                phrase_labels.append(curr_phrase)                           
                curr_phrase = label[2:]                                     
        elif tag=='L':                                                      
            if curr_phrase != label[2:]:                                    
                # Add if the previous label is different                    
                phrase_labels.append(curr_phrase)                           
            # Add current label                                             
            phrase_labels.append(label[2:])                                 
            curr_phrase = ''                                                
        elif tag=='O':                                                      
            # Do nothing other than pushing the previous label              
            if curr_phrase:                                                 
                phrase_labels.append(curr_phrase)                           
            curr_phrase = ''                                                
        elif tag=='U':                                                      
            if curr_phrase:                                                 
                phrase_labels.append(curr_phrase)                           
            phrase_labels.append(label[2:])                                 
        else:                                                               
            print('Tag is incorrect in: {0}.'.format(label))                
            try:                                                            
                assert(False)                                               
            except:                                                         
                pdb.set_trace()
    if curr_phrase!='':
        phrase_labels.append(curr_phrase)
    phrase_labels = [leave_one_word(leave_one_word(phrase_label, 'left_identifier'), 'right_identifier')\
                     for phrase_label in phrase_labels]
    return phrase_labels

In [73]:
source_building = 'ebu3b'
target_building = 'ebu3b'

N = 100
token_type = 'justseparate'
label_type = 'label'
use_cluster_flag = True

# This part should be changed to use MongoDB.
label_filename = 'result/test_result_{0}_{1}_{2}_{3}_{4}_{5}.json'\
            .format(source_building, target_building, token_type, label_type, N,\
                    'clustered' if use_cluster_flag else 'unclustered')
with open(label_filename, 'r') as fp:
    word_label_dict = json.load(fp)

In [74]:
a = word_label_dict['506_2_3015076']

In [78]:
_bilou_phraser(a['sentence'], a['orig_token_labels'])

['network_adapter-nae',
 'left_identifier-06',
 'average',
 'floor',
 'cooling',
 'demand',
 'right_identifier-3rd',
 'floor',
 'average',
 'cooling',
 'right_identifier-3rd',
 'floor',
 'average',
 'cooling',
 'left_identifier-2',
 'building-ebu3b',
 'left_identifier-3b',
 'room',
 'left_identifier-3',
 'right_identifier-3rd',
 'floor',
 'average',
 'cooling',
 'left_identifier-2',
 'average',
 'floor',
 'cooling',
 'left_identifier-2']

In [88]:
''.join(a['sentence'])

'nae 06 programming average floor cooling demand 3rd flr avg clg 3rd flr avg clg pid2\nebu3b.rm-3xxx..3rd flr avg clg-pid2\naverage floor cooling pid2\n'

In [84]:
len(point_tagset_list)

1461

In [86]:
def splitter(s):
    return s.split('_')
point_tagset_list = list(map(splitter, point_tagset_list))

In [87]:
point_tagset_list[0]

['temperature', 'alarm']